PRED RNN MODEL

In [ ]:
import pandas as pd
import os
import cv2
import numpy as np

FRAME_SIZE = 64
SEQUENCE_LENGTH = 10
PREDICT_LENGTH = 5

target_labels = ['BrushingTeeth', 'Biking', 'PushUps','PullUps', 'BenchPress', 'JumpingJack']

train_filtered_df = df_train[df_train['label'].isin(target_labels)]
train_filtered_df.head(5)




,clip_name,clip_path,label
665,v_BenchPress_g14_c03,/train/BenchPress/v_BenchPress_g14_c03.avi,BenchPress
666,v_BenchPress_g05_c06,/train/BenchPress/v_BenchPress_g05_c06.avi,BenchPress
667,v_BenchPress_g21_c03,/train/BenchPress/v_BenchPress_g21_c03.avi,BenchPress
668,v_BenchPress_g23_c05,/train/BenchPress/v_BenchPress_g23_c05.avi,BenchPress
669,v_BenchPress_g25_c01,/train/BenchPress/v_BenchPress_g25_c01.avi,BenchPress


In [ ]:
df_train['label'].unique()

array(['Swing', 'SkyDiving', 'BreastStroke', 'TableTennisShot',
       'FloorGymnastics', 'Rowing', 'SoccerJuggling', 'BenchPress',
       'TennisSwing', 'FrisbeeCatch', 'Kayaking', 'ApplyLipstick',
       'RopeClimbing', 'JumpingJack', 'BlowingCandles', 'BabyCrawling',
       'HammerThrow', 'BoxingPunchingBag', 'Skiing', 'UnevenBars',
       'PullUps', 'RockClimbingIndoor', 'HeadMassage', 'HulaHoop',
       'Rafting', 'ShavingBeard', 'LongJump', 'FieldHockeyPenalty',
       'VolleyballSpiking', 'Punch', 'Archery', 'PlayingDhol',
       'CleanAndJerk', 'PlayingDaf', 'GolfSwing', 'PlayingSitar',
       'IceDancing', 'SkateBoarding', 'BodyWeightSquats',
       'HandstandPushups', 'BalanceBeam', 'WallPushups', 'HorseRiding',
       'Bowling', 'JumpRope', 'CuttingInKitchen', 'CliffDiving',
       'PlayingGuitar', 'FrontCrawl', 'HandstandWalking', 'Fencing',
       'Basketball', 'Typing', 'TaiChi', 'PlayingCello', 'Lunges',
       'Shotput', 'Nunchucks', 'ApplyEyeMakeup', 'BaseballPitch',
 

In [ ]:
train_filtered_df['label'].unique()

array(['BenchPress', 'JumpingJack', 'PullUps', 'PushUps', 'Biking',
       'BrushingTeeth'], dtype=object)

In [ ]:
from google.colab import drive
import os
import pandas as pd

drive.mount('/content/drive', force_remount=True)


train_base_path = '/content/drive/MyDrive/train'


if os.path.exists(train_base_path):
    print(f"Train folder found at '{train_base_path}'.")
else:
    print(f"Train folder not found at '{train_base_path}'. Please check the path.")


train_filtered_df['clip_path'] = train_filtered_df.apply(
    lambda row: os.path.join(train_base_path, row['label'], row['clip_name'] + '.avi'), axis=1
)

print("\nVerifying train paths:")
for idx, row in train_filtered_df.iterrows():
    exists = os.path.exists(row['clip_path'])
    print(f"{row['clip_path']} - Exists: {exists}")


Mounted at /content/drive
Train folder found at '/content/drive/MyDrive/train'.

Verifying train paths:
/content/drive/MyDrive/train/BenchPress/v_BenchPress_g14_c03.avi - Exists: True
/content/drive/MyDrive/train/BenchPress/v_BenchPress_g05_c06.avi - Exists: True
/content/drive/MyDrive/train/BenchPress/v_BenchPress_g21_c03.avi - Exists: True
/content/drive/MyDrive/train/BenchPress/v_BenchPress_g23_c05.avi - Exists: True
/content/drive/MyDrive/train/BenchPress/v_BenchPress_g25_c01.avi - Exists: True
/content/drive/MyDrive/train/BenchPress/v_BenchPress_g03_c04.avi - Exists: True
/content/drive/MyDrive/train/BenchPress/v_BenchPress_g08_c05.avi - Exists: True
/content/drive/MyDrive/train/BenchPress/v_BenchPress_g02_c06.avi - Exists: True
/content/drive/MyDrive/train/BenchPress/v_BenchPress_g24_c03.avi - Exists: True
/content/drive/MyDrive/train/BenchPress/v_BenchPress_g15_c03.avi - Exists: True
/content/drive/MyDrive/train/BenchPress/v_BenchPress_g05_c07.avi - Exists: True
/content/drive/M

<ipython-input-20-152c0bb53394>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_filtered_df['clip_path'] = train_filtered_df.apply(


/content/drive/MyDrive/train/PullUps/v_PullUps_g05_c02.avi - Exists: True
/content/drive/MyDrive/train/PullUps/v_PullUps_g12_c01.avi - Exists: True
/content/drive/MyDrive/train/PullUps/v_PullUps_g18_c02.avi - Exists: True
/content/drive/MyDrive/train/PullUps/v_PullUps_g15_c03.avi - Exists: True
/content/drive/MyDrive/train/PullUps/v_PullUps_g16_c04.avi - Exists: True
/content/drive/MyDrive/train/PullUps/v_PullUps_g04_c02.avi - Exists: True
/content/drive/MyDrive/train/PullUps/v_PullUps_g19_c04.avi - Exists: True
/content/drive/MyDrive/train/PullUps/v_PullUps_g16_c02.avi - Exists: True
/content/drive/MyDrive/train/PullUps/v_PullUps_g10_c01.avi - Exists: True
/content/drive/MyDrive/train/PullUps/v_PullUps_g04_c04.avi - Exists: True
/content/drive/MyDrive/train/PullUps/v_PullUps_g19_c01.avi - Exists: True
/content/drive/MyDrive/train/PullUps/v_PullUps_g05_c04.avi - Exists: True
/content/drive/MyDrive/train/PullUps/v_PullUps_g23_c03.avi - Exists: True
/content/drive/MyDrive/train/PullUps/v

In [ ]:
train_df = train_filtered_df

In [ ]:
train_df

,clip_name,clip_path,label
665,v_BenchPress_g14_c03,/content/drive/MyDrive/train/BenchPress/v_Benc...,BenchPress
666,v_BenchPress_g05_c06,/content/drive/MyDrive/train/BenchPress/v_Benc...,BenchPress
667,v_BenchPress_g21_c03,/content/drive/MyDrive/train/BenchPress/v_Benc...,BenchPress
668,v_BenchPress_g23_c05,/content/drive/MyDrive/train/BenchPress/v_Benc...,BenchPress
669,v_BenchPress_g25_c01,/content/drive/MyDrive/train/BenchPress/v_Benc...,BenchPress
...,...,...,...
9193,v_BrushingTeeth_g03_c05,/content/drive/MyDrive/train/BrushingTeeth/v_B...,BrushingTeeth
9194,v_BrushingTeeth_g21_c04,/content/drive/MyDrive/train/BrushingTeeth/v_B...,BrushingTeeth
9195,v_BrushingTeeth_g23_c06,/content/drive/MyDrive/train/BrushingTeeth/v_B...,BrushingTeeth
9196,v_BrushingTeeth_g16_c01,/content/drive/MyDrive/train/BrushingTeeth/v_B...,BrushingTeeth


PRED RNN

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.utils import Sequence
from tensorflow.keras.models import load_model
from skimage.metrics import structural_similarity as ssim

device = '/GPU:0' if tf.config.list_physical_devices('GPU') else '/CPU:0'
print(f"Using device: {device}")

#parameters same for all
FRAME_HEIGHT, FRAME_WIDTH = 64, 64
NUM_INPUT_FRAMES = 10
NUM_PREDICT_FRAMES = 5
BATCH_SIZE = 4
EPOCHS = 40
LEARNING_RATE = 0.01


def ssim_loss(y_true, y_pred):
    return 1 - tf.reduce_mean(tf.image.ssim(y_true, y_pred, max_val=1.0))

def combined_loss(y_true, y_pred):
    return ssim_loss(y_true, y_pred) + tf.reduce_mean(tf.keras.losses.MAE(y_true, y_pred))

#rnn model
class PredRNN(tf.keras.Model):
    def __init__(self, input_shape, num_predict_frames, **kwargs):
        super(PredRNN, self).__init__(**kwargs)
        self.input_shape = input_shape
        self.num_predict_frames = num_predict_frames
        self.encoder = models.Sequential([
            layers.ConvLSTM2D(64, (3, 3), padding='same', return_sequences=True),
            layers.BatchNormalization(),
            layers.Dropout(0.2)
        ])
        self.decoder = models.Sequential([
            layers.ConvLSTM2D(64, (3, 3), padding='same', return_sequences=True),
            layers.BatchNormalization(),
            layers.Dropout(0.2),
        ])
        self.output_layer = layers.Conv3D(3, (3, 3, 3), padding='same', activation='sigmoid')

    def call(self, inputs):
        encoded = self.encoder(inputs)
        decoded = self.decoder(encoded)
        return self.output_layer(decoded[:, :self.num_predict_frames])

    def get_config(self):
        config = super(PredRNN, self).get_config()
        config.update({
            'input_shape': self.input_shape,
            'num_predict_frames': self.num_predict_frames
        })
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)


class VideoDataLoader(Sequence):
    def __init__(self, dataframe, batch_size):
        self.dataframe = dataframe
        self.batch_size = batch_size
        self.indices = np.arange(len(self.dataframe))

    def __len__(self):
        return int(np.ceil(len(self.dataframe) / self.batch_size))

    def __getitem__(self, idx):
        batch_indices = self.indices[idx * self.batch_size: (idx + 1) * self.batch_size]
        batch_clips = self.dataframe.iloc[batch_indices]
        input_frames, output_frames = [], []

        for _, row in batch_clips.iterrows():
            video_path = row['clip_path']
            cap = cv2.VideoCapture(video_path)
            frames = []

            # Read all frames from the video
            while True:
                ret, frame = cap.read()
                if not ret:
                    break
                frame = cv2.resize(frame, (FRAME_WIDTH, FRAME_HEIGHT)) / 255.0
                frames.append(frame)
            cap.release()


            if len(frames) >= NUM_INPUT_FRAMES + NUM_PREDICT_FRAMES:

                input_frame_indices = []
                remaining_frames = len(frames) - NUM_PREDICT_FRAMES
                idx = 0

                while len(input_frame_indices) < NUM_INPUT_FRAMES:
                    input_frame_indices.append(idx)
                    if len(input_frame_indices) % 2 == 1:
                        idx += 6  # skip 5 logic
                    else:
                        idx += 1

                    if idx >= remaining_frames:
                        input_frame_indices = list(range(remaining_frames - NUM_INPUT_FRAMES, remaining_frames))

                input_frames.append([frames[i] for i in input_frame_indices])
                output_frames.append(frames[NUM_INPUT_FRAMES:NUM_INPUT_FRAMES + NUM_PREDICT_FRAMES])

        return np.array(input_frames), np.array(output_frames)

# for the model
input_shape = (NUM_INPUT_FRAMES, FRAME_HEIGHT, FRAME_WIDTH, 3)
model = PredRNN(input_shape, NUM_PREDICT_FRAMES)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
              loss=combined_loss, metrics=['mae'])

train_loader = VideoDataLoader(train_df, BATCH_SIZE)


with tf.device(device):
    history = model.fit(train_loader, epochs=EPOCHS)




Using device: /GPU:0
Epoch 1/40


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


141/141 ━━━━━━━━━━━━━━━━━━━━ 448s 3s/step - loss: 0.5852 - mae: 0.1414
Epoch 2/40
141/141 ━━━━━━━━━━━━━━━━━━━━ 39s 251ms/step - loss: 0.2308 - mae: 0.0571
Epoch 3/40
141/141 ━━━━━━━━━━━━━━━━━━━━ 38s 257ms/step - loss: 0.2117 - mae: 0.0476
Epoch 4/40
141/141 ━━━━━━━━━━━━━━━━━━━━ 39s 270ms/step - loss: 0.1938 - mae: 0.0433
Epoch 5/40
141/141 ━━━━━━━━━━━━━━━━━━━━ 37s 251ms/step - loss: 0.1952 - mae: 0.0435
Epoch 6/40
141/141 ━━━━━━━━━━━━━━━━━━━━ 41s 253ms/step - loss: 0.1841 - mae: 0.0401
Epoch 7/40
141/141 ━━━━━━━━━━━━━━━━━━━━ 38s 253ms/step - loss: 0.1868 - mae: 0.0416
Epoch 8/40
141/141 ━━━━━━━━━━━━━━━━━━━━ 41s 255ms/step - loss: 0.1635 - mae: 0.0346
Epoch 9/40
141/141 ━━━━━━━━━━━━━━━━━━━━ 42s 268ms/step - loss: 0.1710 - mae: 0.0369
Epoch 10/40
141/141 ━━━━━━━━━━━━━━━━━━━━ 40s 258ms/step - loss: 0.1665 - mae: 0.0351
Epoch 11/40
141/141 ━━━━━━━━━━━━━━━━━━━━ 38s 261ms/step - loss: 0.1702 - mae: 0.0364
Epoch 12/40
141/141 ━━━━━━━━━━━━━━━━━━━━ 39s 265ms/step - loss: 0.1605 - mae: 0.0340
Ep

In [ ]:

model.save("PredRNN.keras")


load it

In [ ]:

loaded_model = load_model("model05.keras", custom_objects={
    'PredRNN': PredRNN,
    'combined_loss': combined_loss,
    'ssim_loss': ssim_loss
})

print("Model loaded successfully.")


Model loaded successfully.


In [ ]:
loaded_model.summary()



Model: "pred_rnn_21"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ sequential_70 (Sequential)           │ (None, 10, 64, 64, 64)      │         154,880 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential_71 (Sequential)           │ (None, 10, 64, 64, 64)      │         295,424 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3d_35 (Conv3D)                   │ (None, 5, 64, 64, 3)        │           5,187 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,365,963 (5.21 MB)

 Trainable params: 455,235 (1.74 MB)

 Non-trainable params: 256 (1.00 KB)

 Optimizer params: 910,472 (3.47 MB)